In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from statistics import mean
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


In [ ]:
df=pd.read_csv('/content/Superstore Sales.csv', encoding='latin-1')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/08/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/08/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,06/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [ ]:
df1=df.copy()

In [ ]:
df_original=df.copy()

In [ ]:
df.shape

(9994, 21)

In [ ]:
df['Ship Date']=pd.to_datetime(df['Ship Date'])
df['Ship Date'].head()

,Ship Date
0,2016-11-11
1,2016-11-11
2,2016-06-16
3,2015-10-18
4,2015-10-18


In [ ]:
df['Order Date']=pd.to_datetime(df['Order Date'])

In [ ]:
def typ(x):
    if x<0.00:
        x=1
    elif x>0.00:
        x=0
    else:
        x=2
    return x

In [ ]:
df['Profit']=df['Profit'].apply(lambda x: typ(x))
df['Profit'].dtypes

dtype('int64')

In [ ]:
df['Order ID'].value_counts()

,count
Order ID,
CA-2017-100111,14
CA-2017-157987,12
CA-2016-165330,11
US-2016-108504,11
CA-2015-131338,10
...,...
CA-2016-157259,1
CA-2017-107986,1
US-2015-112508,1


In [ ]:
X=df.drop('Profit', axis=1)
y=df['Profit']

In [ ]:
X_enc=X.copy()
for col in X.columns:
    le = LabelEncoder()
    X_enc[col]=le.fit_transform(X[col].values)

X_enc.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount
0,0,2500,864,929,2,143,166,0,0,194,15,217,2,12,0,4,386,4205,1,0
1,1,2500,864,929,2,143,166,0,0,194,15,217,2,55,0,5,839,5219,2,0
2,2,2296,732,787,2,237,201,1,0,266,3,517,3,946,1,10,1433,969,1,0
3,3,4372,519,568,3,705,687,0,0,153,8,170,2,319,0,16,366,5392,4,7
4,4,4372,519,568,3,705,687,0,0,153,8,170,2,1316,1,14,573,1360,1,3


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_enc,y, test_size=0.3)

In [ ]:
dt=DecisionTreeClassifier(criterion='gini', max_depth=9)
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=9)

In [ ]:
dt.score(X_train,y_train)

0.9741243745532523

In [ ]:
dt.score(X_test, y_test)

0.9469823274424808

In [ ]:
y_pred1=dt.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred1))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      2410
           1       0.90      0.84      0.87       568
           2       0.33      0.10      0.15        21

    accuracy                           0.95      2999
   macro avg       0.73      0.64      0.66      2999
weighted avg       0.94      0.95      0.94      2999



In [ ]:
accuracy_score(y_test, y_pred1)

0.9469823274424808

random forest

In [ ]:
rf=RandomForestClassifier(100)
rf.fit(X_train, y_train)
RandomForestClassifier()
rf.score(X_train,y_train), rf.score(X_test,y_test)
(1.0, 0.9449816605535178)
y_pred2=rf.predict(X_test)
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      2410
           1       0.94      0.78      0.85       568
           2       0.00      0.00      0.00        21

    accuracy                           0.94      2999
   macro avg       0.63      0.59      0.61      2999
weighted avg       0.94      0.94      0.94      2999



In [ ]:
paramgrid_rf = {'max_depth': list(range(1, 20, 2)), 'n_estimators': list(range(1, 200, 20))}
grid_search_rf=GridSearchCV(RandomForestClassifier(random_state=1),paramgrid_rf)
grid_search_rf.fit(X_train,y_train)
rf_best=grid_search_rf.best_estimator_
rf_best

RandomForestClassifier(max_depth=11, n_estimators=101, random_state=1)

In [ ]:
sort_X=X_enc[['Discount', 'Product Name', 'Sub-Category', 'Sales', 'Ship Mode', 'City']]
sort_X.head()

,Discount,Product Name,Sub-Category,Sales,Ship Mode,City
0,0,386,4,4205,2,194
1,0,839,5,5219,2,194
2,0,1433,10,969,2,266
3,7,366,16,5392,3,153
4,3,573,14,1360,3,153


In [ ]:
sort_X_test=X_test[['Discount', 'Product Name', 'Sub-Category', 'Sales', 'Ship Mode', 'City']]
sort_X_test.head()

,Discount,Product Name,Sub-Category,Sales,Ship Mode,City
4020,4,1226,5,3713,3,78
9604,0,606,1,3880,0,77
3157,0,1073,0,1510,3,329
4952,0,301,1,2707,2,438
3542,4,1450,5,3112,0,94


In [ ]:
i=1
score_rf=[]
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
for train_index,test_index in kf.split(sort_X,y):
    #print(train_index,test_index)
    print('\n{} of kfold {}'.format(i,kf.n_splits))
    xtr,xvl = sort_X.loc[train_index],sort_X.loc[test_index]
    ytr,yvl = y[train_index],y[test_index]
    model = rf_best
    model.fit(xtr, ytr)
    pred_test = model.predict(xvl)
    score = accuracy_score(yvl,pred_test)
    print('accuracy_score',score)
    score_rf.append(score)
    i+=1

print('\n\nAverage Accuracy we can achieve is: ', mean(score_rf))


1 of kfold 5
accuracy_score 0.9469734867433717

2 of kfold 5
accuracy_score 0.9514757378689345

3 of kfold 5
accuracy_score 0.9409704852426213

4 of kfold 5
accuracy_score 0.9559779889944973

5 of kfold 5
accuracy_score 0.950950950950951


Average Accuracy we can achieve is:  0.9492697299600751


additional